In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import os
import re

__Problem:__

Websites with relevant information sometimes 'comment out' their data. This makes the data inaccessible via traditional scraping methods.

This will be demonstrated by trying to scrape the relief pitching statistics for the 2017 Major League Baseball (MLB) season. We will first try via Beautiful Soup and then use Selenium, which is a remote web browser that can be launched using Python.

Lets first try to get table with Pandas read_html method. The read_html method has a parameter 'match' that is passed in as string / regex. Tables containing this string are returned. If no string is used then all tables (if they exist) will be returned. It should be noted that the pandas_html function returns a list equal to the number of tables found, this is true even if 1 table is found.

The table that we are trying to scrape is the 2nd one on the page. We are going use 'Age' as the match parameter as it is one of the headers in the 2nd table and is not contained in the 1st table.

In [2]:
url = 'https://www.baseball-reference.com/leagues/MLB/2017-reliever-pitching.shtml'

In [3]:
req0 = requests.get(url=url)
soup = BeautifulSoup(req0.text, 'lxml')
tbl0 = soup.select('table')

In [4]:
try:
    df0 = pd.read_html(str(tbl0), match='Age')
    if len(df0) == 1:
        df0 = df0[0]
        print('Only one table found and it has {} rows'.format(len(d)))
    else:
        print('more than one table available, try using the match parameter')
except ValueError as v:
    print('Error encountered: {}'.format(v))

Error encountered: No tables found matching pattern 'Age'


As is shown by the error, no tables were found that contained 'Age'. To make sure that we are not in error I will drop the 'Age' term which effectively tells the function to output all the tables on the url. From visual inspection we know that there are at least 2 tables available. Therefore the list that is returned should be at least of length 2.

In [5]:
df1 = pd.read_html(str(tbl0))
if len(df1) == 1:
    df1 = df1[0]
    print('Only one table found and it has {} rows'.format(len(df1)))
else:
    print('more than one table available, try using the match parameter')

Only one table found and it has 32 rows


This returns only one table and it is quite small leading us to believe that it found only the 1st table which consists of team by team relief statistics. Let's make sure by viewing the first and last 3 rows of the returned dataframe.

In [6]:
df1.head(3)

,Tm,RA/G,G,GR,GF,Wgr,Lgr,SVOpp,SV,BSv,...,Tie,Bhd,Runr,Empt,>3o,<3o,IPmult,0DR,Out/GR,Pit/GR
0,ARI,4.07,162,514,160,27,18,63,43,20,...,69,185,162,352,97,186,120,116,2.9,16
1,ATL,5.07,162,530,162,26,18,59,36,23,...,69,256,122,408,91,140,98,101,3.0,16
2,BAL,5.19,162,492,161,30,18,53,35,18,...,60,243,170,322,153,119,180,93,3.6,20


In [7]:
df1.tail(3)

,Tm,RA/G,G,GR,GF,Wgr,Lgr,SVOpp,SV,BSv,...,Tie,Bhd,Runr,Empt,>3o,<3o,IPmult,0DR,Out/GR,Pit/GR
29,WSN,4.15,162,487,159,25,18,64,46,18,...,56,170,122,365,79,148,96,93,2.9,16
30,LgAvg,4.65,162,522,160,26,24,60,39,21,...,68,217,145,377,114,142,132,100,3.2,18
31,NaN,4.65,4860,15659,4801,790,731,1794,1179,615,...,2048,6509,4362,11297,3427,4267,3967,3011,3.2,18


As we suspected this is table is team by team statistics which is not what we need. Maybe using Beautiful soup will be able to find the data needed. Below is code to find all the table rows and headers from the url.

Try BeautifulSoup by finding all rows in url.

In [8]:
dTR = soup.find_all('tr')
m = [[a.get_text() for a in dTR[i].find_all('td')]
     for i in range(len(dTR))]
col0_ = [z.get_text().lower() for z in soup.find_all('th')]
col1_ = [[k.get_text().lower() for k in dTR[j].find_all('th')]
         for j in range(len(dTR))]

print('This code found {} rows'.format(len(m)))

This code found 34 rows


34 rows seems similiar to the number of rows found by the read_html function. Lets print out the top 3 and bottom 3 rows to see if it is the same table as before:

In [9]:
pd.DataFrame(m, columns=col1_[0][1:]).head(3)

,ra/g,g,gr,gf,wgr,lgr,svopp,sv,bsv,sv%,...,tie,bhd,runr,empt,>3o,<3o,ipmult,0dr,out/gr,pit/gr
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,4.07,162,514,160,27,18,63,43,20,68%,...,69,185,162,352,97,186,120,116,2.9,16
2,5.07,162,530,162,26,18,59,36,23,61%,...,69,256,122,408,91,140,98,101,3.0,16


In [10]:
pd.DataFrame(m, columns=col1_[0][1:]).tail(3)

,ra/g,g,gr,gf,wgr,lgr,svopp,sv,bsv,sv%,...,tie,bhd,runr,empt,>3o,<3o,ipmult,0dr,out/gr,pit/gr
31,4.65,162,522,160,26,24,60,39,21,66%,...,68,217,145,377,114,142,132,100,3.2,18
32,4.65,4860,15659,4801,790,731,1794,1179,615,66%,...,2048,6509,4362,11297,3427,4267,3967,3011,3.2,18
33,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


This is the same table minus some nice formatting. Lets move to using a remote web browser given that we did not find the desired table using these traditional nethods. The remote web browser we will be using is Selenium.

__Assumptions:__

* Python 2.7 or higher
* basic Python knowledge
* basic knowledge of HTML

Software Preliminaries:

Download the webdriver that is congruent with the webbrowser you are going to be using. I will be using the Google Chrome webdriver which is freely available at the following url:

* https://sites.google.com/a/chromium.org/chromedriver/downloads

Make sure that you know where the webdriver is stored on your local machine as Selenium needs this information in order to open a remote browser. To keep it simple, I created a folder named 'web_drivers' within my Documents folder and saved the Google Chrome webdriver there.

Now that the webdriver has been downloaded Selenium needs to be installed. You can first check to see if Selenium is installed by opening up the terminal / command line and writing 'pip freeze' or 'conda list' if you are using the Python version distrubuted by Continum Analytics. Assuming that it is not installed, from the terminal write pip install Selenium. After a moment Selenium should be installed.

__Preliminaries:__

We need to determine how the data is stored on the website of interest. Prior to any web scraping, navigate to the website of interest. Within the data that is needed right click and choose inspect. This should display a window showing HTML code. We now have three options to tell Selenium where the data is located within the html: 

* use the element class
* use the element id
* use Xpath

I usually choose the one of the last two if they are available. Why? Element IDs are unique and element classes are not, so if an HTML element has an ID you know that it is the only one within the website. Using Xpath is somewhat easy because it will systematically find the data needed. However, I only use Xpath through the inspect window (see below) because I do not have enough real Xpath experience to feel confident in writing my own.

To find the Xpath through the inspect window follow these steps:

1. Find the html tag where the data is stored and click on it. You should see three dots to the left.
2. Click on the dots and hover over copy and then select Copy Xpath. This will copy the xpath to the clipboard.

Lets now go through the steps to scrape some data using Selenium.

__Step 1:__
Store two variables
* variable 1 --> location/path of webdriver
* variable 2 --> url of the website that has the relevant data

Fire up a remote browser and navigate to the web page where the data can be found.

In [11]:
wd_location = '/Users/jonathanarmitage/Documents/web_drivers/chromedriver'
# wd_location = 'path/to/webdriver/'
url = 'https://www.baseball-reference.com/leagues/MLB/2017-reliever-pitching.shtml'

__Step 2:__

* Load the remote browser and navigate to the url of the page whose data is to be scraped.

* We are not using a phantom browser, so you should see a new instance of a web broswer load. This is the browser that we are going to control.

In [12]:
# load remote browser
browser = webdriver.Chrome(executable_path=wd_location)

# use browser instance to navigate to url
browser.get(url)

Since Selenium is a remote browser it will not find things that are not on the page. Therefore, it is good to maximize the viewing window. If a button or check box is to be toggled, then run the 'window scroll' function so that the toggle of interest is viewable by Selenium.

In [15]:
# browser.maximize_window()
browser.execute_script("window.scrollTo(0, 1000);")

Step 3:

Store the element identifier into a variable. I am going to use Xpath as the identifier for this tutorial, just remember to store it as a string.

In [16]:
xpathID = '//*[@id="players_reliever_pitching"]'

Use the browser instance to find the element located in the XpathID variable

In [17]:
elem0 = browser.find_element_by_xpath(xpathID)

Locate table of interest using Pandas read_html method utilizing the 'match' parmaeter. There are rows that have no data so we will remove any rows where 'Name' is in the column 'Name'. We will also remove the colymn 'Rk' since it is of no interest.

In [18]:
tables = pd.read_html(str(browser.page_source), match='Age')[0]
tables = tables.loc[tables['Name'] != 'Name'].reset_index(drop=True)
tables.drop('Rk', axis=1, inplace=True)

Find all unique urls using a list comprehension:

In [19]:
getIDs = [x.get_attribute('href') for x in elem0.find_elements_by_tag_name('a')]

Compile regex to locate first part of each observations unique id:

In [20]:
reX = re.compile('https://www.baseball-reference.com/players/\w/')

Loop through 'getIDs' and find instances that only have '/players/' in them:

In [21]:
pl0 = [] 
for i in getIDs:
    if '/players/' in i:
        x1 = re.sub(reX, '', i).replace('.shtml', '')
        pl0.append(x1)

Format instances and columns

In [22]:
tables['Name'] = tables['Name'].str.replace('\xa0', ' ').str.replace('*', '').str.strip()

In [23]:
tables.columns = tables.columns.str.lower().str.replace('/', '_').str.replace('>', 'more_than_').\
str.replace('<', 'less_than_').str.replace('-', '_').str.replace('≥', 'more_than_or_equal').\
str.replace('%', '_pct')

tables.rename(columns={'g': 'games', 'w': 'wins', 'l': 'losses'}, inplace=True)

In [24]:
tables['sv_pct'] = tables['sv_pct'].str.replace('%', '').str.strip()
tables['is_pct'] = tables['is_pct'].str.replace('%', '').str.strip()

In [25]:
static = ['name', 'bbrefID', 'tm', 'year']
notStatic = [x for x in tables if x not in static]

for col_ in notStatic:
    tables[col_] = tables[col_].astype(float)

If length of playerIDs equals length of dataframe --> add playerIDs to dataframe and then close the browser
Else print 'Length mismatch' and close the remote browser

In [26]:
if len(pl0) == len(tables):
    tables['bbrefID'] = pl0
    browser.close()
else:
    print('length mismatch')
    browser.close()

View top 3 rows of dataframe

In [27]:
tables.head()

,name,age,tm,ip,games,gr,gf,wgr,lgr,svopp,...,bhd,runr,empt,more_than_3o,less_than_3o,ipmult,0dr,out_gr,pit_gr,bbrefID
0,Fernando Abad,31.0,BOS,43.2,48.0,48.0,15.0,2.0,1.0,3.0,...,32.0,17.0,31.0,8.0,26.0,12.0,6.0,2.7,16.0,abadfe01
1,Austin Adams,26.0,WSN,5.0,6.0,6.0,3.0,0.0,0.0,0.0,...,5.0,0.0,6.0,0.0,1.0,0.0,0.0,2.5,22.0,adamsau02
2,Tim Adleman,29.0,CIN,122.1,30.0,10.0,4.0,0.0,2.0,0.0,...,6.0,2.0,8.0,3.0,1.0,4.0,1.0,5.1,30.0,adlemti01
3,Andrew Albers,31.0,SEA,41.0,9.0,3.0,2.0,1.0,0.0,1.0,...,1.0,1.0,2.0,3.0,0.0,3.0,0.0,9.7,56.0,alberan01
4,Matt Albers,34.0,WSN,61.0,63.0,63.0,23.0,7.0,2.0,6.0,...,20.0,26.0,37.0,13.0,22.0,17.0,16.0,2.9,15.0,alberma01


Encapsulate all of the above code into a function. I have added a parameter that allows the user to scrape either reliever or starter statistics. No code changed except the url that is called.

In [28]:
def getData(pos_type, mlb_season):
    
    """
    Control remote web browser do scrape starter and reliever atatistics
    
    Parameters:
    -----------
    pos_type: str
        options: {'rel', 'starter'}
    mlb_season: int
    
    Returns:
    --------
    Dataframe
    """
    
    wd_location = 'path/to/webdriver'
#     wd_location = '/Users/jonathanarmitage/Documents/web_drivers/chromedriver'
    
    if pos_type == 'starter':
        url = 'https://www.baseball-reference.com/leagues/MLB/{}-starter-pitching.shtml'.format(mlb_season)
        xpathID = '//*[@id="players_starter_pitching"]'
    elif pos_type == 'rel':
        url = 'https://www.baseball-reference.com/leagues/MLB/{}-reliever-pitching.shtml'.format(mlb_season)
        xpathID = '//*[@id="players_reliever_pitching"]'

    browser = webdriver.Chrome(executable_path=wd_location)

    # use browser instance to navigate to url
    browser.get(url)
    browser.execute_script("window.scrollTo(0, 1000);")
    
    elem0 = browser.find_element_by_xpath(xpathID)
    
    # Locate table of interest using Pandas read_html method
    #     Remove all rows that include 'Name'
    #     Remove 'Rk' column
    tables = pd.read_html(str(browser.page_source), match='Age')[0]
    tables = tables.loc[tables['Name'] != 'Name'].reset_index(drop=True)
    tables.drop('Rk', axis=1, inplace=True)
    
    # obtain playerIDs
    getIDs = [x.get_attribute('href') for x in elem0.find_elements_by_tag_name('a')]
    
    reX = re.compile('https://www.baseball-reference.com/players/\w/')
    
    pl0 = [] 
    for i in getIDs:
        if '/players/' in i:
            x1 = re.sub(reX, '', i).replace('.shtml', '')
            pl0.append(x1)
    
    # clean data
    tables['Name'] = tables['Name'].str.replace('\xa0', ' ').str.replace('*', '').str.strip()
    tables.columns = tables.columns.str.lower().str.replace('/', '_').str.replace('>', 'more_than_').\
    str.replace('<', 'less_than_').str.replace('-', '_').str.replace('≥', 'more_than_or_equal').\
    str.replace('%', '_pct')

    tables.rename(columns={'g': 'games', 'w': 'wins', 'l': 'losses'}, inplace=True)
    tables['year'] = mlb_season
    
    # conditional cleaning based on pos_type
    if pos_type == 'starter':
        tables['qs_pct'] = tables['qs_pct'].str.replace('%', '').str.strip()
    elif pos_type == 'rel':
        tables['sv_pct'] = tables['sv_pct'].str.replace('%', '').str.strip()
        tables['is_pct'] = tables['is_pct'].str.replace('%', '').str.strip()

    static = ['name', 'bbrefID', 'tm', 'year']
    notStatic = [x for x in tables if x not in static]

    for col_ in notStatic:
        tables[col_] = tables[col_].astype(float)
    
    # conditional calculations based on pos_type
    if pos_type == 'starter':
        tables['qs_pct'] = np.where(tables['gs'] > 0, tables['qs'] / tables['gs'], np.nan)
    elif pos_type == 'rel':
        tables['sv_pct'] = np.where(tables['svopp'] > 0, tables['sv'] / tables['svopp'], np.nan)
        tables['is_pct'] = np.where(tables['ir'] > 0, tables['is'] / tables['ir'], np.nan)

    if len(pl0) == len(tables):
        tables['bbrefID'] = pl0
        browser.close()
        return tables
    else:
        print('length mismatch')
        browser.close()

Since some observations play for multiple teams it is necessary to condense there statistics. Here is a function that does this:

In [29]:
def trimMult(X):

    """
    Splits dataframe, X, into two seperate dataframes:
        - observations who appeared with a single team, tbl1
        - observations that appeared with multiple teams, tbl2
    Dataframe tbl2 is parsed to include only 1 row per observation
    The dataframes are then row-wise concatanated
    
    Parameters:
    -----------
    X: dataframe
        holds scraped observations from the function getData()
        
    Returns:
    --------
    Dataframe
    """
    grp0 = pd.DataFrame(X.groupby(['name', 'bbrefID']).size().reset_index()).rename(columns={0: 'freq'})
    tbl0 = pd.merge(X, grp0, on=['name', 'bbrefID'], how='left')
    
    tbl1 = tbl0.loc[tbl0['freq'] == 1].reset_index(drop=True)
    tbl2 = tbl0.loc[(tbl0['freq'] >= 2) & (tbl0['tm'] == 'TOT')].reset_index(drop=True)
    
    tbl3 = pd.concat([tbl1, tbl2], axis=0).reset_index(drop=True)
    tbl3.drop('freq', axis=1, inplace=True)
    
    # formats innings column
    tbl3['ip0'] = tbl3['ip'].astype(str).str.split('.').str[0].astype(int)
    tbl3['ip1'] = tbl3['ip'].astype(str).str.split('.').str[1].astype(int) / 3

    tbl3['ip2'] = tbl3['ip0'] + tbl3['ip1']

    tbl3.drop(['ip', 'ip0', 'ip1'], axis=1, inplace=True)
    tbl3.rename(columns={'ip2': 'ip'}, inplace=True)
    
    return tbl3

In [46]:
A = getData(pos_type='starter', mlb_season=2003)

In [47]:
F = trimMult(X=A)

To scrape multiple seasons the above function can be nested in a for loop. The iterator being the year to scrape. 

In [ ]:
holdDF = pd.DataFrame()
for year in range(2010, 2013):
    print('Working on MLB season: {}'.format(year))
    tmpDF0 = getData(pos_type='rel', mlb_season=year)
    tmpDF1 = trimMult(X=tmpDF0)
    holdDF = holdDF.append(tmpDF1)

In conclusion this tutorial has shown us how to access a remote browser and scrape data from it.